## Notebook for aligning phase images in a tilt series

### 0. Global parameter definitions
### 1. Measuring miss-tilt and finding rotation axis
### 2. Correcting sample drift
### 3. Creating 3-D mask
#### Imports

In [4]:
%matplotlib qt

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle
import os

In [6]:
import fpd
import hyperspy.api as hs
import skimage.filters as skfl

In [8]:
import pyramid as pr
import mbir.alignment as pa
import mbir.util as pu
import mbir.reconstruction as pre

### 0. Global parameter definitions
#### Define where the DataSet objects with each phase image are located

In [9]:
#file paths

folders = ['+0',
 '+20',
 '+30',
 '-10',
 '-20',
 '-30',
 '-40',
 '-50',
 '-60']

# file names for single (phasemap + projector + mask) datasets
f_name="data.pickle"

In [10]:
#load all the phase images
no_projections=len(folders)
datas=[None]*no_projections

for i in range(len(datas)):
    data_path=folders[i]+'\\'+f_name
    with open(data_path, 'rb') as f:
        datas[i]=pickle.load(f)

In [11]:
#plot masks and phasemaps
masks=[]
phasemaps_raw=[]
for i in range(len(datas)):
    datas[i].plot_phasemaps()
    phasemap=datas[i].phasemaps[0]
    phasemaps_raw.append(phasemap)
    print(f"phasemap {i} shape", phasemap.dim_uv)
    
phasemaps=pa.pad_equalise_tilt_series(phasemaps_raw)
    
for pm in phasemaps:
    mask=pm.mask
    masks.append(mask)

phasemap 0 shape (120, 116)
phasemap 1 shape (120, 116)
phasemap 2 shape (120, 116)
phasemap 3 shape (120, 116)
phasemap 4 shape (120, 116)
phasemap 5 shape (120, 116)
phasemap 6 shape (120, 116)
phasemap 7 shape (120, 116)
phasemap 8 shape (120, 116)
Pixel spacing is 10.266 +/- 5.99e-07


### 1. Measuring miss-tilt and finding rotation axis
#### Start with approximate alignment of sample position

In [12]:
#Before measuring the projection angles, shift all phase images such that the sample is in approximately the same position.

# method is "cross_correlation" or "centre_of_mass"
method="centre_of_mass"
# index of the reference image all others should be compared to and shifted to match
test_image_index=0


shifts = pa.find_image_shifts(masks, method=method)
phasemaps_similar = pa.pad_translate_tilt_series(phasemaps, shifts)
projectors=[data.projectors[0] for data in datas]

#### Define a region where the symmetry axis of a nanowire can be measured

In [13]:
#fit a line to each mask and determine wire angle. Also extract x-tilt values from projectors.

crop_right=80
crop_left=25
crop_bottom=80
crop_top=0

tana, xtilts = pa.measure_wire_orientations(phasemaps_similar, projectors, crop_right=crop_right, crop_left=crop_left, 
                                         crop_bottom=crop_bottom, crop_top=crop_top, plot_results=True, verbose=False)

#### Based on measurements of symmetry axis direction, estimate the orientation of the axis of rotation (th0), and the starting miss-tilt (p0)

In [14]:
# define an error function that goes to 0 if the correct axis of rotation is identified 
# Method is true for an arbitrarily orientated volume of revolution (e.g. cylinders and cones).
#methods = 'L-BFGS-B' or 'TNC'
# lowest 'distance from 0' is best

tilt_axis_direction_guess=np.radians(15)
miss_tilt_guess=np.radians(0)

sol, fun = pa.minimise_total_error(tana, xtilts, x0 = [tilt_axis_direction_guess, miss_tilt_guess], method='TNC', 
                                   error_window_width=np.radians(3)) 
th0, p0 = sol.x
#print(sol)

root theta: 9.55 deg +6.242 / -6.242
root x_tilt: 6.47 deg +1.297 / -1.297
distance from 0 error: 0.0020326497804954305


In [15]:
#inspect the error function
pa.plot_3D_surface(fun)

#zoom in on the solution in the error function
range_rad=np.radians(3)

bracket = [[-range_rad+th0, range_rad+th0],[-range_rad+p0, range_rad+p0]]
pa.plot_3D_surface(fun, bracket=bracket, title="Error function (zoomed in)")

In [ ]:
#### remeasure the sample symmetry axis after correction have been applied.

In [23]:
#refine tilt and direction measurements
#rotates the images to have tilt axis be horizontal and accounts for miss-tilt

crop_right=70
crop_left=30
crop_bottom=80
crop_top=0


tana_ref, xtilts_ref = pa.measure_wire_orientations(phasemaps_similar, projectors, theta=th0, xtilt0=p0, 
                                                    crop_right=crop_right, crop_left=crop_left, 
                                         crop_bottom=crop_bottom, crop_top=crop_top, plot_results=True, verbose=False)


#### apply the measured axis of rotation to correct the phase maps

In [25]:
#align the phasemaps

#identify the location of the wire tip
tip_axis=1 # 0 -> y_axis, 1 -> x_axis #which direction should the wire tip stands out.
use_high_end=True # right side of the image if the high end of axis 1.

#adjust to select the wanted feature on the flattest lying image
crop_right=70
crop_left=30
crop_bottom=80
crop_top=0



tilts = np.degrees(xtilts_ref)
axis_rot = np.degrees(th0)
phasemap_titles=["projection at %.1f deg"%tilt for tilt in tilts]

#rotate the phasemaps such that x-axis coresponds to tilt axis
print("\nrotating phasemaps")
phasemaps_rot=pa.rotate_phasemaps(phasemaps_similar, axis_rot, smooth_masks=True)
 

#centre the phase images such that the confidence region centres coincide
print("\ncentring phasemaps") 
phasemaps_centered = pa.centre_phasemaps(phasemaps_rot, padded=False)


#measure translations of the needle tip along the x-axis and correct
print("\nx-shifting phasemaps")
phasemaps_translated = pa.align_wire_tips(phasemaps_centered, axis=tip_axis, use_high_end=use_high_end, padded=False, verbose=True)

#trim unnecessary empty space
print("\ntrimming some empty space from phasemaps")
phasemaps_trim = pa.trim_empty_space(phasemaps_translated, equal_trim=True)

#calculate projections of a starting image and shift all others to match.
print("\naligning phasemap projections")
phasemaps_corrected, reconstruction_dimensions = pa.align_wire_directions(phasemaps_trim, tilts, plot_fits=True, plot_aligned_masks=True, crop_right=crop_right,
                        crop_left=crop_left, crop_top=crop_top, crop_bottom=crop_bottom,
                        test_mask_index=None, use_round_projection=False, 
                        axis=1, z_ang=0, subcount=5, padded=False, verbose=True)


#trim unnecessary empty space
print("\ntrimming all unnecessary empty space from phasemaps")
phasemaps_aligned = pa.trim_empty_space(phasemaps_corrected, equal_trim=False, verbose=False)

print("\nfinished alignments")


rotating phasemaps

centring phasemaps

x-shifting phasemaps
shifts (index, (dy,dx)): [(0, (0, -3)), (1, (0, 5)), (2, (0, 4)), (3, (0, 0)), (4, (0, -1)), (5, (0, 1)), (6, (0, -2)), (7, (0, -1)), (8, (0, -4))]

trimming some empty space from phasemaps

aligning phasemap projections
0 tilt mask is calculated from mask 3
Starting projector calculation for tilt 6.47 deg
Starting projector calculation for tilt 26.47 deg
Starting projector calculation for tilt 36.47 deg
Starting projector calculation for tilt -3.53 deg
Starting projector calculation for tilt -13.53 deg
Starting projector calculation for tilt -23.53 deg
Starting projector calculation for tilt -33.53 deg
Starting projector calculation for tilt -43.53 deg
Starting projector calculation for tilt -53.53 deg
projector calculations finished

Mask 0 Y-shift = 3, Wire direction error = -1.22 deg
Mask 1 Y-shift = 0, Wire direction error = -0.42 deg
Mask 2 Y-shift = 5, Wire direction error = -0.75 deg
Mask 3 Y-shift = 0, Wire directio

#### Inspect aligned phase maps

In [32]:
plot_slice=slice(None)
t=[pm.plot_phase() for pm in phasemaps_aligned[plot_slice]]

In [30]:
# Size of 3D volume needed to accomodate all of the geometric 3D model.
reconstruction_dimensions

(47, 117, 107)

### 2. Creating 3-D mask
#### Backproject a subset of the tilt series to determine what 3D volume dimensions are necessary.

In [31]:
#test if the 3-D mask is created correctly when using a small number of phasemaps

dimz, dimy, dimx = reconstruction_dimensions
dim=(5,dimy,dimx) # reduce the dimension to make the code run faster

selection=slice(0,1)
z_rotation=0 #deg

no_slices=len(phasemaps_aligned)
z_rots=([z_rotation]*no_slices)
camera_rots=([camera_rotation]*no_slices)
pixel_spacing= phasemaps_aligned[0].a




data_test=pa.make_projection_data(phasemaps_aligned[selection], z_rots[selection], tilts[selection], camera_rots[selection], 
                               pixel_spacing, dim=dim, plot_results=False)

data_test.plot_mask(title="3d mask")

Reconstruction voxel number: 312975
Pixel size: 10.2657 nm
3d reconstructions dimensions: (25, 117, 107)
starting projector calculation
1/1; projector calculation finished



#### Backproject all phasemaps to create a 3D model

In [38]:
#now calculate projectors for all phasemaps
dim=(21,dimy-20,dimx-20) # reduce dimensiont to make the calculation faster. Must have odd numbers.
extra_tilt = -6 #deg. if we want to intentionally introduce miss-tilt to make the sample lie flat.

x_tilts = tilts + extra_tilt

data_series=pa.make_projection_data(phasemaps_aligned, z_rots, x_tilts, camera_rots, 
                               pixel_spacing, dim=dim, plot_results=False)

data_series.plot_mask(title="raw mask")

Reconstruction voxel number: 177219
Pixel size: 10.2657 nm
3d reconstructions dimensions: (21, 97, 87)
starting projector calculation
1/9; 2/9; 3/9; 4/9; 5/9; 6/9; 7/9; 8/9; 9/9; projector calculation finished



#### By comparing to SEM images we can identify the missing wedge artefact in the geometric model. To correct it, we can edit mask manually in MS paint.

In [43]:
mask_projection_axis=2  # 012 <-> zyx
data_series.set_3d_mask(threshold=1)
fname=pa.save_editable_mask(data_series.mask, axis=mask_projection_axis)
print(fname)

png shape (21, 97, 4)
mask_projection.png


In [44]:
data_series.mask = pa.load_png_mask(data_series.mask, "mask_projection - Copy.png", axis=mask_projection_axis)
data_series.plot_mask(title="cropped mask")

.png shape (21, 97, 4)
mask shape (21, 97, 87)


#### refine the mask

In [47]:
#find best 3d mask smoothing
mask_threshold = 0.45 # cut-off threshold for redefining the mask
sigma_mask=1 #gaussian filter standard deviation

mask3d_0 = data_series.mask.copy()
mask3d = np.where(data_series.mask, 1.0, 0.0)
mask3d = skfl.gaussian(mask3d, sigma = sigma_mask)
mask3d = np.where(mask3d > mask_threshold, True, False)
pu.matshow_n([np.sum(data_series.mask,axis=0), np.sum(mask3d,axis=0)],["old mask","smooth mask"] )
data_series.mask = mask3d
data_series.plot_mask(title="smoothed mask")
data_series.mask = mask3d_0

In [48]:
#save the best smoothed mask
data_series.mask = mask3d


#### Save aligned data series

In [50]:
with open(r"results\data_series.pickle", "wb") as f:
    pickle.dump(data_series, f)

In [51]:
# inspect the phasemaps
data_series.plot_phasemaps()

### Quick reconstruction
Unlikely to produce quantitative results, but can be used for inspection.

In [53]:
with open("results\data_series.pickle", "rb") as f:
    data_series_r=pickle.load(f)
mask0 = data_series.mask.copy()

In [55]:
magdata_rec, cost_fun = pre.reconstruct_from_phasemaps_simple(data_series_r, lam=1e-2, verbose=True, max_iter=200)

CG:   0%|          | 0/200 [00:00<?, ?it/s]

#### End of notebook. Once the tilt series is aligned and a geometric 3D model is defined, see the magnetisation recontruction notebook for a quantitative reconstruction.